In [2]:
import pandas as pd
import time
from tqdm import tqdm
from uuid import uuid4

from pinecone import Pinecone, ServerlessSpec
from langchain_openai import OpenAIEmbeddings

import os
from dotenv import load_dotenv
load_dotenv()

c:\Users\Playdata\anaconda3\envs\vectordb\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
INDEX_NAME = "funeral-services"

In [ ]:
# 각 파일별로 'type' 지정 및 컬럼 매핑 정의
file_configs = [
    {
        "filename": "cemeteries.csv",
        "type": "cemetery", # 묘지
        "mapping": {"지역": "region", "소재지": "location", "시설명": "name", "주소": "address", "연락처": "phone"}
    },
    {
        "filename": "colubarium.csv",
        "type": "columbarium", # 봉안당
        "mapping": {"지역": "region", "소재지": "location", "시설명": "name", "주소": "address", "연락처": "phone"}
    },
    {
        "filename": "crematoriums.csv",
        "type": "crematorium", # 화장시설
        "mapping": {"지역": "region", "소재지": "location", "시설명": "name", "주소": "address", "연락처": "phone"}
    },
    {
        "filename": "natural_burial.csv",
        "type": "natural_burial", # 자연장지
        "mapping": {"지역": "region", "소재지": "location", "시설명": "name", "주소": "address", "연락처": "phone"}
    },
    {
        "filename": "funeral_homes.csv",
        "type": "funeral_home", # 장례식장
        "mapping": {"소재지": "region", "장례식장 이름": "name", "주소": "address", "전화번호": "phone"}
        # 장례식장은 'location'(구/군) 컬럼이 없으므로 주소에서 추출 예정
    }
]

def load_and_unify_data(configs):
    combined_df = pd.DataFrame()
    
    for config in configs:
        try:
            # 파일 읽기 (인코딩 에러 처리)
            try:
                df = pd.read_csv(config["filename"])
            except UnicodeDecodeError:
                df = pd.read_csv(config["filename"], encoding='cp949')
            
            # 컬럼명 변경
            df = df.rename(columns=config["mapping"])
            
            # type 컬럼 추가
            df['type'] = config["type"]
            
            # 장례식장 파일 등 'location'이 없는 경우 주소에서 추출 (예: "서울 종로구..." -> "종로구")
            if 'location' not in df.columns:
                df['location'] = df['address'].apply(
                    lambda x: x.split()[1] if isinstance(x, str) and len(x.split()) > 1 else ""
                )
            
            # 필요한 컬럼만 선택 및 결측치 처리
            target_cols = ['type', 'region', 'location', 'name', 'address', 'phone']
            
            # 데이터프레임에 없는 컬럼이 있다면 빈 값으로 생성
            for col in target_cols:
                if col not in df.columns:
                    df[col] = ""
            
            df = df[target_cols].fillna("정보없음")
            
            # 검색에 사용될 'text' 필드 생성 (임베딩 대상)
            # 예: "시설유형: 장례식장, 시설명: 서울대병원, 위치: 서울 종로구..."
            df['text'] = df.apply(
                lambda row: f"시설유형: {row['type']}, 지역: {row['region']} {row['location']}, 시설명: {row['name']}, 주소: {row['address']}, 전화번호: {row['phone']}",
                axis=1
            )
            
            combined_df = pd.concat([combined_df, df], ignore_index=True)
            print(f"[OK] {config['filename']} 로드 완료 ({len(df)}건)")
            
        except Exception as e:
            print(f"[Error] {config['filename']} 처리 중 오류: {e}")
            
    return combined_df

# ---------------------------------------------------------
# 3. Pinecone 적재 로직
# ---------------------------------------------------------
def upload_to_pinecone(df):
    # Pinecone 초기화
    pc = Pinecone(api_key=PINECONE_API_KEY)
    
    # 인덱스 확인 및 생성 (없을 경우)
    if INDEX_NAME not in pc.list_indexes().names():
        pc.create_index(
            name=INDEX_NAME,
            dimension=1536, # OpenAI text-embedding-3-small 기준
            metric='cosine',
            spec=ServerlessSpec(cloud='aws', region='us-east-1')
        )
    
    index = pc.Index(INDEX_NAME)
    embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY, model="text-embedding-3-small")
    
    batch_size = 100
    total_rows = len(df)
    
    print(f"\n총 {total_rows}개의 데이터를 Pinecone에 적재합니다...")
    
    for i in tqdm(range(0, total_rows, batch_size)):
        batch = df.iloc[i:i+batch_size]
        
        # 1. 텍스트 임베딩 생성
        texts = batch['text'].tolist()
        ids = [str(uuid4()) for _ in range(len(batch))]
        embeds = embeddings.embed_documents(texts)
        
        # 2. 메타데이터 준비
        # Pinecone은 메타데이터 값을 문자열, 숫자, 불리언, 리스트만 허용함
        metadatas = batch.drop(columns=['text']).to_dict(orient='records')
        
        # text 필드도 메타데이터에 포함하여 RAG 시 원문 참조 가능하게 함
        for j, meta in enumerate(metadatas):
            meta['text'] = texts[j]
            
        # 3. 벡터 데이터 구성 (id, vector, metadata)
        vectors = list(zip(ids, embeds, metadatas))
        
        # 4. 업로드
        index.upsert(vectors=vectors)
        
    print("적재 완료!")

# ---------------------------------------------------------
# 실행
# ---------------------------------------------------------
if __name__ == "__main__":
    # 1. 데이터 로드 및 전처리
    unified_df = load_and_unify_data(file_configs)
    
    # 2. 데이터 확인
    print("\n--- 데이터 예시 ---")
    print(unified_df.head(2))
    
    # 3. Pinecone 적재 (API 키 설정 후 주석 해제하여 실행)
    # upload_to_pinecone(unified_df)